# Working with Time Zones

## Aware datetime comparison semantics

We will start this section off with a mysterious bug involving the comparison of datetimes:

In [1]:
from datetime import datetime, timedelta
from dateutil import tz

In [2]:
LON = tz.gettz('Europe/London')

# Construct a datetime
x = datetime(2007, 3, 25, 1, 9, tzinfo=LON)
ts = x.timestamp()    # Get a timestamp representing the same datetime

# Get the same datetime from the timestamp
y = datetime.fromtimestamp(ts, LON)

# Get thesame datetime from the timestamp with a fresh instance of LON
z = datetime.fromtimestamp(ts, tz.gettz.nocache('Europe/London'))

In [3]:
x == y

False

In [4]:
x == z

True

In [5]:
y == z

True

To summarize: x, y and z should all represent the same `datetime` – they all have the same time zone, and y and z are the result of converting x into a timestamp and then back into a `datetime`, but for some reason `x != y`, and, even more curiously, `x == z`, even though the only difference between y and z is that z uses a different `tzinfo` object (representing the same zone). Even stranger, the equality relationship between the three is non-transitive, because `x != y` even though `x == z` and `y == z`. What the hell is going on?

### Inter-zone vs. intra-zone comparisons

In Python, comparisons between datetimes are divided into "same zone" and "different zone" comparisons. When two `datetime`s are in the same time-zone, they are equal if their "wall time" is the same, *even if they represent different absolute times*:

In [6]:
dt1 = datetime(2017, 10, 29, 1, 30, tzinfo=LON)
dt2 = datetime(2017, 10, 29, 1, 30, fold=1, tzinfo=LON)

print(f"{dt1} | {dt1.timestamp()}")
print(f"{dt2} | {dt2.timestamp()}")

2017-10-29 01:30:00+01:00 | 1509237000.0
2017-10-29 01:30:00+00:00 | 1509240600.0


In [7]:
dt1 == dt2

True

Comparisons between datetimes in *different* zones, however, use "absolute time" semantics, meaning that the wall time can be different, as long as they refer to the same time in UTC:

In [8]:
dt1 = datetime(2018, 4, 7, 1, 30, tzinfo=tz.gettz("America/Chicago"))
dt2 = datetime(2018, 4, 7, 2, 30, tzinfo=tz.gettz("America/New_York"))
print(f"{dt1} | {dt1.timestamp()}")
print(f"{dt2} | {dt2.timestamp()}")

2018-04-07 01:30:00-05:00 | 1523082600.0
2018-04-07 02:30:00-04:00 | 1523082600.0


In [9]:
dt1 == dt2

True

#### What does it mean to be in the "same zone"?

When deciding what is the "same zone" and what is a "different zone", Python uses *object equality* rather than *value equality*, which means to say two datetimes are in the same zone if `dt1.tzinfo is dt2.tzinfo`, and otherwise they are in different zones, *even if* `dt1.tzinfo == dt2.tzinfo`!

This means that we can re-cast the "intra-zone" comparison as an "inter-zone" comparison by just getting a new object to represent the same zone:

In [10]:
dt1 = datetime(2017, 10, 29, 1, 30, tzinfo=LON)
dt2 = datetime(2017, 10, 29, 1, 30, fold=1, tzinfo=tz.gettz.nocache("Europe/London"))

print(f"{dt1} | {dt1.timestamp()}")
print(f"{dt2} | {dt2.timestamp()}")
print("")
print(f"{dt1} == {dt2}: {dt1 == dt2}")

2017-10-29 01:30:00+01:00 | 1509237000.0
2017-10-29 01:30:00+00:00 | 1509240600.0

2017-10-29 01:30:00+01:00 == 2017-10-29 01:30:00+00:00: False


### Solving the mystery

Returning to the mystery we started the section off with, we can now apply the same/different zone comparison semantics to our non-transitive equality and see what's happening.

In [11]:
def zone_semantics(dt1, dt2):
    return "Same Zone" if dt1.tzinfo is dt2.tzinfo else "Different Zone"

print(f"x == y ({zone_semantics(x, y)}): {x == y}")
print(f"x: {x} ({x.timestamp()})")
print(f"y: {y} ({y.timestamp()})")
print("")
print(f"x == z ({zone_semantics(x, z)}): {x == z}")
print(f"x: {x} ({x.timestamp()})")
print(f"z: {z} ({z.timestamp()})")
print("")
print(f"y == z ({zone_semantics(y, z)}): {y == z}")
print(f"y: {y} ({y.timestamp()})")
print(f"z: {z} ({z.timestamp()})")

x == y (Same Zone): False
x: 2007-03-25 01:09:00+01:00 (1174781340.0)
y: 2007-03-25 00:09:00+00:00 (1174781340.0)

x == z (Different Zone): True
x: 2007-03-25 01:09:00+01:00 (1174781340.0)
z: 2007-03-25 00:09:00+00:00 (1174781340.0)

y == z (Different Zone): True
y: 2007-03-25 00:09:00+00:00 (1174781340.0)
z: 2007-03-25 00:09:00+00:00 (1174781340.0)


#### But wait...
Aren't `y` and `z` supposed to be `datetime.fromtimestamp(x.timestamp())`? Why is the wall time different after a round trip through a timestamp?

And if these are all in the exact same time zone, why are there two different wall times for the same timestamp?

In [12]:
tz.datetime_exists(x)

False

... because `x` is an imaginary datetime!

## Aware datetime arithmetic semantics

In [13]:
import tz_answers
from tz_answers import wall_add, wall_sub
from tz_answers import absolute_add, absolute_sub
from tz_answers import AbsoluteDateTime, WallDateTime

We have looked at *comparison* semantics, but it is worth noting that the difference between "same zone" and "different zone" comparisons really only makes a difference during *ambiguous* times and as we've seen in *imaginary* times. At all other times, the two are equivalent.

Comparison is just a specific case of the more general arithmetical property of `datetime` objects that operations between datetimes in the same zone use "wall time" semantics (i.e. looking at the clock on the wall) and operations between dateimes in different zones use "absolute time" semantics (i.e. looking at a stopwatch or timestamp).


#### Wall time vs. absolute time semantics
Colloquially, time periods tend to be overloaded concepts, and what you mean when you say, e.g., "a day" or "a month" depends on the context. Looking at the following code, what would you expect the value for `dt2` to be?

In [15]:
NYC = tz.gettz("America/New_York")
dt1 = datetime(2018, 3, 10, 13, tzinfo=NYC)
dt2 = dt1 + timedelta(days=1)

There are two options, the first is using "wall time" semantics, returning the next day at the same time (rolling the clock forward by 24 hours):

In [16]:
print(dt1)
print(wall_add(dt1, timedelta(days=1)))

2018-03-10 13:00:00-05:00
2018-03-11 13:00:00-04:00


The second option is to use "absolute time" semantics, where we jump forward to the point in time after which 24 hours have elapsed in the "real world". Because the start date here is immediately before a daylight saving time transition, these give *different* answers:

In [17]:
print(dt1)
print(absolute_add(dt1, timedelta(days=1)))

2018-03-10 13:00:00-05:00
2018-03-11 14:00:00-04:00


The concept of "add one day" is overloaded in this situation because the meaning of "1 day" can either mean "the period between two identical clock times on subsequent days" or it can mean "the period during which 24 hours have elapsed".

**Question**: Which behavior is more intuitive? What would you choose for the default?

Now look at subtraction between two datetimes across a DST transition:

In [18]:
dt1 = datetime(2018, 3, 10, 13, 30, tzinfo=NYC)
dt2 = datetime(2018, 3, 11, 8, 30, tzinfo=NYC)

In [19]:
print(wall_sub(dt2, dt1))

19:00:00


In [20]:
print(absolute_sub(dt2, dt1))

18:00:00


A total of 18 real hours have elapsed, but `8 - 13 % 24 == 19`, so 19 "wall hours" have elapsed.

**Question**: Which behavior is more intuitive? What would you choose as the default (ignoring your choice for the earlier question)


### Python's behavior

Assuming Python wanted to be consistent and have all operations use *either* wall-time operations or absolute-time operations, whatever you choose is likely to have some non-intuitive behaviors. If, for example, we were to choose to always use *absolute* `datetime`s, you would likely find someone who writes some variation of this code:

In [21]:
DAY = timedelta(days=1)
dtstart = AbsoluteDateTime(2018, 3, 9, 12, tzinfo=NYC)
for i in range(4):
    print(dtstart + i * DAY)

2018-03-09 12:00:00-05:00
2018-03-10 12:00:00-05:00
2018-03-11 13:00:00-04:00
2018-03-12 13:00:00-04:00


and they would be confused as to why all of a sudden their noon meeting was happening at 1 PM after a DST transition. If you choose *wall* times for everything, you'll get someone who writes some variation on *this* code:

In [22]:
dtstart = WallDateTime(2018, 4, 17, 12, tzinfo=NYC)
dtend = WallDateTime(2018, 4, 17, 12, tzinfo=tz.gettz('America/Los_Angeles'))

In [23]:
print(dtstart)
print(dtend)

2018-04-17 12:00:00-04:00
2018-04-17 12:00:00-07:00


In [24]:
print(dtend - dtstart)

0:00:00


and they'll be surprised to find that these two events that happened 3 hours apart give 0 hours difference!

As we saw in the comparison case, Python actually uses hybrid semantics that can be somewhat confusing, though there is at least a rhyme and a reason behind it. For arithmetic within the "same zone", all operations use wall time semantics. Because addition returns a `datetime` in the same time zone as the input `datetime`, it is always considered a "same zone" operation, and thus always uses wall time semantics.

Because wall time semantics across different time zones is essentially *never* what you want, the choice of "absolute time" semantics for subtractions between two datetimes in different time zones also makes sense and gives you the "intuitive" behavior you would expect.

However, note that both wall-time subtraction and absolute-time subtraction return a `timedelta`, which has no notion of whether it represents a period of "wall time" or a period of "absolute time". As a result, you are stuck with a counter-intuitive result:

In [25]:
dt1 = datetime(2018, 3, 11, 1, tzinfo=NYC)
dt2 = datetime(2018, 3, 11, 1, tzinfo=tz.gettz('America/Los_Angeles'))

In [26]:
dt1 == dt2 + (dt1 - dt2)

True

In [27]:
dt2 == dt1 + (dt2 - dt1)

False

Across a DST boundary, you cannot reverse a between-zone boundary by adding the result (a `timedelta`) to the original value!

### Exercise: Implement explicit wall-time and absolute-time arithmetic


Knowing how Python handles aware time zone arithmetic, by paying attention to the time zones you work in you should be able to design your system so that you get the semantics you want, *but* it is occasionally the case that you will mostly want wall-time semantics but for some operations you want absolute time, or vice-versa. In this case, you will want a way to explicitly operate using either wall-time or absolute-time semantics.

In this exercise, implement the `wall_add`, `wall_sub`, `absolute_add` and `absolute_sub` functions that explicitly give you wall time or absolute time semantics.

In [28]:
def wall_add(dt: datetime, offset: timedelta) -> datetime:
    """Addition with "wall-time" semantics"""
    pass

### Uncomment to test
# tz_tests.test_wall_add(wall_add)

In [29]:
def wall_sub(dt: datetime, other: datetime) -> timedelta:
    """Subtraction with "wall time" semantics"""
    pass

### Uncomment to test
# tz_tests.test_wall_sub(wall_sub)

In [30]:
def absolute_add(dt: datetime, offset: timedelta) -> datetime:
    """Addition with "absolute time" semantics"""
    pass

### Uncomment to test
# tz_tests.test_absolute_add(absolute_add)

In [31]:
def absolute_sub(dt: datetime, other: datetime) -> timedelta:
    """Subtraction with "absolute time" semantics"""
    pass

### Uncomment to test
# tz_tests.test_absolute_sub(absolute_sub)